In [1]:
# imports
import cv2
import numpy as np
import mediapipe as mp
import pandas as pd
import time
import matplotlib.pyplot as plt
%matplotlib inline
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

In [2]:
# Capturing landmarks
correct_landmarks = {"correct_left_hand": [], "correct_right_hand": [], "correct_pose": []}
timeout = 10

In [3]:
workout_time = time.time() + timeout + 10   # 30 seconds from now (10 seconds for the user to get ready)
cap = cv2.VideoCapture(0)
#recording video
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
size = (frame_width, frame_height)
record = cv2.VideoWriter('expected.avi', cv2.VideoWriter_fourcc(*'MJPG'), 10, size)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make Detections
        results = holistic.process(image)

        # Recolor image back to BGR for rendering
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(121, 44, 250), thickness=2, circle_radius=2)
                                  )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(121, 44, 250), thickness=2, circle_radius=2)
                                  )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                  )
        # storing landmarks
        # try:
        #     # # Right hand
        #     # right_hand = results.right_hand_landmarks
        #     # # right_hand_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in right_hand.landmark]).flatten())
        #     # if(right_hand is not None):
        #     #     correct_right_hand = np.append(correct_right_hand, right_hand)
        #     # # Left Hand
        #     # left_hand = results.left_hand_landmarks
        #     # # left_hand_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in left_hand.landmark]).flatten())
        #     # if(left_hand is not None):
        #     #     correct_left_hand = np.append(correct_left_hand, left_hand)
        #     # # Pose
        #     # pose = results.pose_landmarks
        #     # if pose is not None:
        #     #     correct_pose = np.append(correct_pose, pose)
        # except:
        #     pass
        # Right hand
        right_hand = results.right_hand_landmarks
        # right_hand_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in right_hand.landmark]).flatten())
        correct_landmarks["correct_right_hand"].append(right_hand)
        # Left Hand
        left_hand = results.left_hand_landmarks
        # left_hand_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in left_hand.landmark]).flatten())
        correct_landmarks["correct_left_hand"].append(left_hand)
        # Pose
        pose = results.pose_landmarks
        correct_landmarks["correct_pose"].append(pose)
        # Show to screen
        cv2.imshow('Raw Webcam Feed', image)
        record.write(image)
        # Break gracefully
        if time.time() > workout_time or (cv2.waitKey(10) & 0xFF == ord('q')):
            break
cap.release()
record.release()
cv2.destroyAllWindows()
pd.DataFrame(correct_landmarks).to_csv("correct_landmarks.csv")

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
QObject::moveToThread: Current thread (0x2c32930) is not the object's thread (0x3307840).
Cannot move to target thread (0x2c32930)

QObject::moveToThread: Current thread (0x2c32930) is not the object's thread (0x3307840).
Cannot move to target thread (0x2c32930)

QObject::moveToThread: Current thread (0x2c32930) is not the object's thread (0x3307840).
Cannot move to target thread (0x2c32930)

QObject::moveToThread: Current thread (0x2c32930) is not the object's thread (0x3307840).
Cannot move to target thread (0x2c32930)

QObject::moveToThread: Current thread (0x2c32930) is not the object's thread (0x3307840).
Cannot move to target thread (0x2c32930)

QObject::moveToThread: Current thread (0x2c32930) is not the object's thread (0x3307840).
Cannot move to target thread (0x2c32930)

QObject::moveToThread: Current thread (0x2c32930) is not the object's thread (0x3307840).
Cannot move to target thread (0x2c32930)

QObject::moveToThrea

In [4]:
# testing output
print(len(correct_landmarks["correct_left_hand"]))
print(len(correct_landmarks["correct_right_hand"]))
print(len(correct_landmarks["correct_pose"]))
# sample output
try:
    print(correct_landmarks["correct_left_hand"][0].landmark[0])
    print(correct_landmarks["correct_right_hand"][0].landmark[0])
    print(correct_landmarks["correct_pose"][0].landmark[0])
    print(len(correct_landmarks["correct_pose"][0].landmark) + len(correct_landmarks["correct_left_hand"][0].landmark) + len(correct_landmarks["correct_right_hand"][0].landmark))
except Exception as e:
    print(e)

109
109
109
'NoneType' object has no attribute 'landmark'


In [5]:
def check_landmarks(current, expected, threshold):
    match_count = 0
    if current is None or expected is None:
        return (False, match_count)
    for (current_landmark, expected_landmark) in zip(current.landmark, expected.landmark):
        dist = np.sqrt((current_landmark.x - expected_landmark.x)**2 + (current_landmark.y - expected_landmark.y)**2 + (current_landmark.z - expected_landmark.z)**2)
        if dist < threshold:
            match_count += 1
    if match_count == len(current.landmark):
        return (True, match_count)
    else:
        return (False, match_count)

In [6]:
cap = cv2.VideoCapture(0)
#recording video
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
size = (frame_width, frame_height)
record = cv2.VideoWriter('actual.avi', cv2.VideoWriter_fourcc(*'MJPG'), 10, size)
# setting up counter
count = 0
poses_matched = 0
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make Detections
        results = holistic.process(image)

        match_count = 0
        # matching the landmarks
        threshold = 0.2
        left_hand_matched, right_hand_matched, pose_matched = (False, False, False)
        index = count % len(correct_landmarks["correct_pose"])
        if correct_landmarks["correct_right_hand"][index] is not None:
            (right_hand_matched, right_hand_match_count) = check_landmarks(results.right_hand_landmarks, correct_landmarks["correct_right_hand"][index], threshold)
            match_count += right_hand_match_count
        if correct_landmarks["correct_left_hand"][index] is not None:
            (left_hand_matched, left_hand_match_count) = check_landmarks(results.left_hand_landmarks, correct_landmarks["correct_left_hand"][index], threshold)
            match_count += left_hand_match_count
        if correct_landmarks["correct_pose"][index] is not None:
            (pose_matched, pose_match_count) = check_landmarks(results.pose_landmarks, correct_landmarks["correct_pose"][index], threshold)
            match_count += pose_match_count
        match_percent = match_count / 75
        count += 1
        text = f"Match Percentage: {match_percent} - "
        if match_percent > 0.6:
            poses_matched += 1
            text += "Correct Pose"
            color = (0, 255, 0)
        else: 
            text += "Incorrect Pose"           
            color = (0, 0, 255)
            # drawing the correct pose
            #1. Right hand
            if correct_landmarks["correct_right_hand"][index] is not None:
                mp_drawing.draw_landmarks(image, correct_landmarks["correct_right_hand"][index], mp_holistic.HAND_CONNECTIONS,
                                            mp_drawing.DrawingSpec(color=(51, 255, 51), thickness=2, circle_radius=4),
                                            mp_drawing.DrawingSpec(color=(51, 153, 255), thickness=2, circle_radius=2)
                                            )
            # 2. Left Hand
            if correct_landmarks["correct_left_hand"][index] is not None:
                mp_drawing.draw_landmarks(image, correct_landmarks["correct_left_hand"][index], mp_holistic.HAND_CONNECTIONS,
                                           mp_drawing.DrawingSpec(color=(51, 255, 51), thickness=2, circle_radius=4),     
                                          mp_drawing.DrawingSpec(color=(51, 153,255), thickness=2, circle_radius=2))
            # 3. Pose Detections
            if correct_landmarks["correct_pose"][index] is not None:
                mp_drawing.draw_landmarks(image, correct_landmarks["correct_pose"][index], mp_holistic.POSE_CONNECTIONS,
                                            mp_drawing.DrawingSpec(color=(51, 255, 51), thickness=2, circle_radius=4),
                                            mp_drawing.DrawingSpec(color=(51, 153, 255), thickness=2, circle_radius=2)
                                            )                                                                    
        # Recolor image back to BGR for rendering
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # 1. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=color, thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=color, thickness=2, circle_radius=2)
                                  )

        # 2. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=color, thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=color, thickness=2, circle_radius=2)
                                  )

        # 3. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=color, thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=color, thickness=2, circle_radius=2)
                                  )
        print(f"Left Hand matched:{left_hand_matched} Right Hand matched:{right_hand_matched} Pose matched:{pose_matched}")       
        # Show to screen
        cv2.putText(image, text, (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2, cv2.LINE_AA)
        cv2.imshow('Raw Webcam Feed', image)
        record.write(image)
        # Break gracefully
        if (cv2.waitKey(10) & 0xFF == ord('q')):
            break
cap.release()
record.release()
cv2.destroyAllWindows()
print(f"Total Poses Matched: {poses_matched} out of {count}")
print(f"Accuracy: {poses_matched/count * 100}%")

Left Hand matched:False Right Hand matched:False Pose matched:False
Left Hand matched:False Right Hand matched:False Pose matched:False


QObject::moveToThread: Current thread (0x2c32930) is not the object's thread (0x3307840).
Cannot move to target thread (0x2c32930)

QObject::moveToThread: Current thread (0x2c32930) is not the object's thread (0x3307840).
Cannot move to target thread (0x2c32930)

QObject::moveToThread: Current thread (0x2c32930) is not the object's thread (0x3307840).
Cannot move to target thread (0x2c32930)

QObject::moveToThread: Current thread (0x2c32930) is not the object's thread (0x3307840).
Cannot move to target thread (0x2c32930)

QObject::moveToThread: Current thread (0x2c32930) is not the object's thread (0x3307840).
Cannot move to target thread (0x2c32930)

QObject::moveToThread: Current thread (0x2c32930) is not the object's thread (0x3307840).
Cannot move to target thread (0x2c32930)

QObject::moveToThread: Current thread (0x2c32930) is not the object's thread (0x3307840).
Cannot move to target thread (0x2c32930)

QObject::moveToThread: Current thread (0x2c32930) is not the object's thread

Left Hand matched:False Right Hand matched:False Pose matched:False
Left Hand matched:False Right Hand matched:False Pose matched:False
Left Hand matched:False Right Hand matched:False Pose matched:False
Left Hand matched:False Right Hand matched:False Pose matched:False
Left Hand matched:False Right Hand matched:False Pose matched:False
Left Hand matched:False Right Hand matched:False Pose matched:False
Left Hand matched:False Right Hand matched:False Pose matched:False
Left Hand matched:False Right Hand matched:False Pose matched:False
Left Hand matched:False Right Hand matched:False Pose matched:False
Left Hand matched:False Right Hand matched:False Pose matched:False
Left Hand matched:False Right Hand matched:False Pose matched:False
Left Hand matched:False Right Hand matched:False Pose matched:False
Left Hand matched:False Right Hand matched:False Pose matched:False
Left Hand matched:False Right Hand matched:False Pose matched:False
Left Hand matched:False Right Hand matched:False